In [1]:
# 2014 GoogLeNet(基本模块：Ｉnception)

Inception结构如下：

包含了四条并行线路，四条线图的输出都保持与输入一致．

![GoogLeNet](./5.9_inception.svg)

In [1]:
### Inception

import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append('../d2lzh/')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Inception(nn.Module):
    def __init__(self, in_c, c1, c2, c3, c4):   # c1-c4:四条线路的层的输出通道数
        super(Inception, self).__init__()
        # path 1, 1x1
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        
        # path 2, 1x1 > 3x3
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        
        # path3, 1x1 > 5x5
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        
        # path4, maxpool > 1x1
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)
        
    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(self.p2_1(x)))
        p3 = F.relu(self.p3_2(self.p3_1(x)))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        
        return torch.cat((p1, p2, p3, p4), dim=1) # dim=0是batch,dim=1是channel

In [2]:
### googlenet模型(Inception中各path的filters数目是在ImageNet数据集精细化尝试后确定下来的)

In [7]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

b2 = nn.Sequential(
    nn.Conv2d(64, 64, kernel_size=1),
    nn.Conv2d(64, 192, kernel_size=3, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

b3 = nn.Sequential(
    Inception(192, 64, (96, 128), (16, 32), 32),
    Inception(256, 128, (128, 192), (32, 96), 64),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

b4 = nn.Sequential(
    Inception(480, 192, (96, 208), (16, 48), 64),
    Inception(512, 160, (112, 224), (24, 64), 64),
    Inception(512, 128, (128, 256), (24, 64), 64),
    Inception(512, 112, (144, 288), (32, 64), 64),
    Inception(528, 256, (160, 320), (32, 128), 128),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

b5 = nn.Sequential(
    Inception(832, 256, (160, 320), (32, 128), 128),
    Inception(832, 384, (192, 384), (48, 128), 128),
    d2l.GlobalAvgPool2d(),
)

net = nn.Sequential(
    b1, b2, b3, b4, b5,
    d2l.FlattenLayer(),
    nn.Linear(1024, 10),
)

In [8]:
# 测试
X = torch.rand(1, 1, 96, 96)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

0 output shape:  torch.Size([1, 64, 24, 24])
1 output shape:  torch.Size([1, 192, 12, 12])
2 output shape:  torch.Size([1, 480, 6, 6])
3 output shape:  torch.Size([1, 832, 3, 3])
4 output shape:  torch.Size([1, 1024, 1, 1])
5 output shape:  torch.Size([1, 1024])
6 output shape:  torch.Size([1, 10])


In [9]:
### 获取数据

In [12]:
batch_size = 64
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

In [13]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 2.4450, train_acc 0.105, test acc 0.100, time 264.4 sec
epoch 2, loss 0.9717, train_acc 0.625, test acc 0.808, time 263.5 sec
epoch 3, loss 0.4494, train_acc 0.834, test acc 0.862, time 263.0 sec
epoch 4, loss 0.3703, train_acc 0.862, test acc 0.873, time 263.2 sec
epoch 5, loss 0.3347, train_acc 0.877, test acc 0.891, time 262.9 sec
epoch 6, loss 0.3099, train_acc 0.884, test acc 0.896, time 263.7 sec
epoch 7, loss 0.2905, train_acc 0.892, test acc 0.900, time 262.6 sec
epoch 8, loss 0.2756, train_acc 0.898, test acc 0.906, time 263.2 sec
epoch 9, loss 0.2626, train_acc 0.902, test acc 0.907, time 265.1 sec
epoch 10, loss 0.2475, train_acc 0.908, test acc 0.914, time 264.6 sec


In [ ]:
# training on  cuda
# epoch 1, loss 2.4450, train_acc 0.105, test acc 0.100, time 264.4 sec
# epoch 2, loss 0.9717, train_acc 0.625, test acc 0.808, time 263.5 sec
# epoch 3, loss 0.4494, train_acc 0.834, test acc 0.862, time 263.0 sec
# epoch 4, loss 0.3703, train_acc 0.862, test acc 0.873, time 263.2 sec
# epoch 5, loss 0.3347, train_acc 0.877, test acc 0.891, time 262.9 sec
# epoch 6, loss 0.3099, train_acc 0.884, test acc 0.896, time 263.7 sec
# epoch 7, loss 0.2905, train_acc 0.892, test acc 0.900, time 262.6 sec
# epoch 8, loss 0.2756, train_acc 0.898, test acc 0.906, time 263.2 sec
# epoch 9, loss 0.2626, train_acc 0.902, test acc 0.907, time 265.1 sec
# epoch 10, loss 0.2475, train_acc 0.908, test acc 0.914, time 264.6 sec